In [237]:
import numpy as np

# Multilevel Monte Carlo Implementation
# Following Giles (2015) algorithm for European option pricing


In [ ]:
### SDE Coefficients

def mu_S(S, M):
    """Drift coefficient for S process"""
    return 0.1 * (np.sqrt(np.minimum(M, S)) - 1) * S

def sigma_S(S):
    """Diffusion coefficient for S process"""
    return 0.5 * S

In [239]:
### MLMC Configuration

T = 1.0              # Time horizon
K = 1.0              # Strike price
M_param = 4.0        # SDE parameter
epsilon = 0.0001     # Target accuracy
max_levels = 10      # Maximum number of levels
N_initial = 10000    # Initial samples per new level


In [ ]:
### MLMC Simulation Functions

def euler_maruyama(N, M, S0, mu_func, sigma_func, M_param, T):
    """
    Euler-Maruyama scheme for SDE simulation
    
    Args:
        N: Number of paths
        M: Number of time steps
        S0: Initial value
        mu_func: Drift function
        sigma_func: Diffusion function
        M_param: SDE parameter
        T: Time horizon
        
    Returns:
        Terminal values S(T) for all paths
    """
    dt = T / M
    DW = np.sqrt(dt) * np.random.randn(N, M)
    S = np.full(N, S0)
    
    for i in range(M):
        mu = mu_func(S, M_param)
        sigma = sigma_func(S)
        S += mu * dt + sigma * DW[:, i]
        S = np.maximum(S, 1e-10)  # Prevent negative values
    
    return S


def mlmc_level(N, M, mu_func, sigma_func, M_param, T, K):
    """
    Simulate single MLMC level (for L=0)
    
    Returns:
        mean: E[payoff]
        var: Var[payoff]
    """
    S_T = euler_maruyama(N, M, 1.0, mu_func, sigma_func, M_param, T)
    payoff = np.maximum(S_T - K, 0)
    return np.mean(payoff), np.var(payoff)


def mlmc_level_diff(N, M_coarse, M_fine, mu_func, sigma_func, M_param, T, K):
    """
    Simulate MLMC level difference (for L>0)
    
    Uses same Brownian motion for coarse and fine paths (telescoping)
    
    Returns:
        mean_diff: E[P_fine - P_coarse]
        var_diff: Var[P_fine - P_coarse]
    """
    dt_fine = T / M_fine
    dt_coarse = T / M_coarse
    ratio = M_fine // M_coarse
    
    # Generate fine Brownian increments
    DW_fine = np.sqrt(dt_fine) * np.random.randn(N, M_fine)
    
    # Aggregate to coarse Brownian (same underlying motion)
    DW_coarse = np.zeros((N, M_coarse))
    for j in range(M_coarse):
        DW_coarse[:, j] = np.sum(DW_fine[:, j*ratio:(j+1)*ratio], axis=1)
    
    # Simulate fine path
    S_fine = np.ones(N)
    for i in range(M_fine):
        mu = mu_func(S_fine, M_param)
        sigma = sigma_func(S_fine)
        S_fine += mu * dt_fine + sigma * DW_fine[:, i]
        S_fine = np.maximum(S_fine, 1e-10)
    
    # Simulate coarse path (same Brownian)
    S_coarse = np.ones(N)
    for i in range(M_coarse):
        mu = mu_func(S_coarse, M_param)
        sigma = sigma_func(S_coarse)
        S_coarse += mu * dt_coarse + sigma * DW_coarse[:, i]
        S_coarse = np.maximum(S_coarse, 1e-10)
    
    # Payoff difference
    P_fine = np.maximum(S_fine - K, 0)
    P_coarse = np.maximum(S_coarse - K, 0)
    diff = P_fine - P_coarse
    
    return np.mean(diff), np.var(diff)


def compute_optimal_samples(variances, M_levels, T, epsilon):
    """
    Compute optimal sample allocation using Giles' formula
    
    N_l = ceil((2/ε²) × √(V_l × h_l) × Σ√(V_j / h_j))
    
    Args:
        variances: List of variances [V_0, ..., V_L]
        M_levels: List of time steps [M_0, ..., M_L]
        T: Time horizon
        epsilon: Target accuracy
        
    Returns:
        List of optimal sample sizes [N_0, ..., N_L]
    """
    h_levels = [T / M for M in M_levels]
    sum_term = sum(np.sqrt(variances[j] / h_levels[j]) for j in range(len(variances)))
    
    N_optimal = []
    for l in range(len(variances)):
        N_l = int(np.ceil((2 / epsilon**2) * np.sqrt(variances[l] * h_levels[l]) * sum_term))
        N_optimal.append(N_l)
    
    return N_optimal

In [ ]:
### MLMC Algorithm (Giles 2015)

print("="*70)
print("MULTILEVEL MONTE CARLO")
print("="*70)

# Initialize
L = 0
M_levels = []
means_sum = []
variances = []
N_current = []
converged = False

while L < max_levels and not converged:
    print(f"\n{'='*70}")
    print(f"LEVEL L = {L}")
    print(f"{'='*70}")
    
    # Step 2: Estimate variance with initial samples
    if L == 0:
        M = 1
        M_levels.append(M)
        print(f"Step 2: Initial sampling (N={N_initial}, M=1)")
        
        mean, var = mlmc_level(N_initial, M, mu_S, sigma_S, M_param, T, K)
        means_sum.append(mean * N_initial)
        variances.append(var)
        N_current.append(N_initial)
        
        print(f"  E[P_0] = {mean:.6f}, Var[P_0] = {var:.6f}")
    else:
        M_coarse = 4**(L-1)
        M_fine = 4**L
        M_levels.append(M_fine)
        
        print(f"Step 2: Initial sampling (N={N_initial}, M_c={M_coarse}, M_f={M_fine})")
        
        mean_diff, var_diff = mlmc_level_diff(N_initial, M_coarse, M_fine, 
                                               mu_S, sigma_S, M_param, T, K)
        means_sum.append(mean_diff * N_initial)
        variances.append(var_diff)
        N_current.append(N_initial)
        
        print(f"  E[P_{L} - P_{L-1}] = {mean_diff:.6f}, Var = {var_diff:.6f}")
    
    # Step 3: Compute optimal samples for all levels
    print(f"\nStep 3: Optimal sample allocation")
    N_optimal = compute_optimal_samples(variances, M_levels, T, epsilon)
    
    for l in range(L+1):
        print(f"  Level {l}: N_opt = {N_optimal[l]:,} (current = {N_current[l]:,})")
    
    # Step 4: Add extra samples as needed
    print(f"\nStep 4: Adding extra samples")
    for l in range(L+1):
        N_extra = N_optimal[l] - N_current[l]
        
        if N_extra > 0:
            print(f"  Level {l}: +{N_extra:,} samples")
            
            if l == 0:
                mean_extra, _ = mlmc_level(N_extra, M_levels[0], mu_S, sigma_S, M_param, T, K)
                means_sum[l] += mean_extra * N_extra
            else:
                M_c = M_levels[l-1]
                M_f = M_levels[l]
                mean_extra, _ = mlmc_level_diff(N_extra, M_c, M_f, mu_S, sigma_S, M_param, T, K)
                means_sum[l] += mean_extra * N_extra
            
            N_current[l] = N_optimal[l]
    
    # Step 5: Test convergence (L ≥ 2)
    if L >= 2:
        mean_L = means_sum[L] / N_current[L]
        threshold = 0.5 * epsilon / np.sqrt(2)
        
        print(f"\nStep 5: Convergence test")
        print(f"  |E[P_{L} - P_{L-1}]| = {abs(mean_L):.6e}")
        print(f"  Threshold = {threshold:.6e}")
        
        if abs(mean_L) < threshold:
            converged = True
            print(f"  ✓ CONVERGED")
    
    # Step 6: Continue to next level
    if not converged:
        L += 1

# Final estimator
print(f"\n{'='*70}")
print("RESULTS")
print(f"{'='*70}")

V_mlmc = sum(means_sum[l] / N_current[l] for l in range(len(means_sum)))
total_samples = sum(N_current)

print(f"\nOption Price: {V_mlmc:.6f}")
print(f"Levels: {len(means_sum)}")
print(f"Total samples: {total_samples:,}")
print(f"Epsilon: {epsilon}")
print(f"\nSample distribution:")
for l in range(len(N_current)):
    print(f"  Level {l}: {N_current[l]:,} samples")
print("="*70)

MULTILEVEL MONTE CARLO

LEVEL L = 0
Step 2: Initial sampling (N=10000, M=1)
  E[P_0] = 0.199706, Var[P_0] = 0.157442

Step 3: Optimal sample allocation
  Level 0: N_opt = 31,488,377 (current = 10,000)

Step 4: Adding extra samples
  Level 0: +31,478,377 samples

LEVEL L = 1
Step 2: Initial sampling (N=10000, M_c=1, M_f=4)
  E[P_1 - P_0] = 0.000849, Var = 0.000106

Step 3: Optimal sample allocation
  Level 0: N_opt = 33,123,335 (current = 31,488,377)
  Level 1: N_opt = 429,963 (current = 10,000)

Step 4: Adding extra samples
  Level 0: +1,634,958 samples
  Level 1: +419,963 samples

LEVEL L = 2
Step 2: Initial sampling (N=10000, M_c=4, M_f=16)
  E[P_2 - P_1] = 0.000809, Var = 0.000081

Step 3: Optimal sample allocation
  Level 0: N_opt = 35,980,607 (current = 33,123,335)
  Level 1: N_opt = 467,052 (current = 429,963)
  Level 2: N_opt = 204,057 (current = 10,000)

Step 4: Adding extra samples
  Level 0: +2,857,272 samples
  Level 1: +37,089 samples
  Level 2: +194,057 samples

Step 5: Co

MemoryError: Unable to allocate 19.5 GiB for an array with shape (10000, 262144) and data type float64